1. Data

1.1 Load

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

X = pd.read_csv('fetal_health.csv', sep = ';')

X.drop(X.tail(1900).index, inplace = True)

X.info()
y = X['fetal_health']
y = y.astype(int)
X = X.drop(columns = ['fetal_health'])
classes = set(y)

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

X = pd.DataFrame(X)

classes

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226 entries, 0 to 225
Data columns (total 22 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   baseline value                                          226 non-null    float64
 1   accelerations                                           226 non-null    float64
 2   fetal_movement                                          226 non-null    float64
 3   uterine_contractions                                    226 non-null    float64
 4   light_decelerations                                     226 non-null    float64
 5   severe_decelerations                                    226 non-null    float64
 6   prolongued_decelerations                                226 non-null    float64
 7   abnormal_short_term_variability                         226 non-null    float64
 8   mean_value_of_short_term_variability    

{1, 2, 3}

1.2 Bootstrap

In [2]:
print(X)
X['y_true'] = y
bts = []
for i in range(3):
  df_bt = X.sample(frac=0.5)
  y_bt = df_bt['y_true']
  X_bt = df_bt.drop(columns=['y_true'])
  bts.append([X_bt, y_bt])


           0         1         2         3    4    5    6         7   \
0    0.133333  0.000000  0.000000  0.000000  0.0  0.0  0.0  0.802817   
1    0.400000  0.352941  0.000000  0.461538  0.2  0.0  0.0  0.014085   
2    0.422222  0.176471  0.000000  0.615385  0.2  0.0  0.0  0.000000   
3    0.444444  0.176471  0.000000  0.615385  0.2  0.0  0.0  0.000000   
4    0.400000  0.411765  0.000000  0.615385  0.0  0.0  0.0  0.000000   
..        ...       ...       ...       ...  ...  ...  ...       ...   
221  0.333333  0.000000  0.038380  0.000000  0.0  0.0  0.0  0.549296   
222  0.333333  0.117647  0.027719  0.000000  0.0  0.0  0.0  0.436620   
223  0.333333  0.352941  0.036247  0.000000  0.0  0.0  0.0  0.394366   
224  0.333333  0.117647  0.034115  0.000000  0.0  0.0  0.0  0.521127   
225  0.333333  0.000000  0.040512  0.000000  0.0  0.0  0.0  0.563380   

           8         9   ...        11        12        13        14   15  \
0    0.000007  0.544304  ...  0.385714  0.110092  0.000000

1.3 Classification

In [3]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

models = [MultinomialNB(), KNeighborsClassifier(), LinearSVC()]
preds= []
for i, model in enumerate(models):
  X_bt = bts[i][0]
  y_bt = bts[i][1]
  model.fit(X_bt, y_bt)
  y_pred = model.predict(X_bt)
  pred_df = pd.DataFrame(y_pred, index = X_bt.index)
  preds.append(pred_df)
  print(accuracy_score(y_bt, y_pred))

0.7699115044247787
0.8495575221238938
0.8584070796460177


1.4 Aggregation of predictions

In [4]:
pred_df = pd.concat(preds, axis=1)
pred_df.head()
pred_df.to_csv('predicts.csv')

2. Ontology

In [5]:
!pip install owlready2
from owlready2 import *

onto = get_ontology('onto.owl')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


2.x

In [6]:
import types

with onto:
  tsr = types.new_class('tsr', (ObjectProperty, TransitiveProperty, SymmetricProperty, ReflexiveProperty))
  ts = types.new_class('ts', (ObjectProperty, TransitiveProperty, SymmetricProperty))

  for cl in classes:
    onto_cl = types.new_class(f'Class{cl}', (Thing,))
    onto_cl_know = types.new_class(f'Know_Class{cl}', (Thing,))
    onto_cl_know.equivalent_to.append(tsr.only(onto_cl))
    onto_cl_believe = types.new_class(f'Believe_Class{cl}', (Thing,))
    onto_cl_believe.equivalent_to.append(ts.only(onto_cl))

onto.save('onto.owl', format='ntriples')

In [7]:
with onto:
  for index, row in pred_df.iterrows():
    inst_old = None
    for j, pred in enumerate(pred_df.columns):
      cl = row.iloc[j]
      if not pd.isna(cl):
        cl = int(cl)
        onto_cl = onto[f'Class{cl}']
        inst_new = onto_cl(f'inst{index}_{j}')
        if inst_old != None:
          inst_old.tsr.append(inst_new)
          inst_old.ts.append(inst_new)
        inst_old = inst_new

onto.save('onto.owl', format='ntriples')

In [8]:
with onto:
  sync_reasoner_pellet(infer_property_values=True, debug = 2)

onto.save('onto_reasoned.owl', format='ntriples')

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /usr/local/lib/python3.8/dist-packages/owlready2/pellet/log4j-1.2.16.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/pellet-2.3.1.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/slf4j-log4j12-1.6.4.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-iri-0.9.5.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/aterm-java-1.6.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/httpcore-4.2.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-core-2.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/xml-apis-1.4.01.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/httpclient-4.2.3.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/antlr-3.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/antlr-runtime-3.2.jar:/usr/local/lib/python3.8/dist-packages

* Owlready * Adding relation onto.inst73_1 tsr onto.inst73_1
* Owlready * Adding relation onto.inst73_1 ts onto.inst73_1
* Owlready * Adding relation onto.inst208_0 tsr onto.inst208_0
* Owlready * Adding relation onto.inst30_1 tsr onto.inst30_1
* Owlready * Adding relation onto.inst30_1 ts onto.inst30_1
* Owlready * Adding relation onto.inst129_1 tsr onto.inst129_1
* Owlready * Adding relation onto.inst106_1 tsr onto.inst106_1
* Owlready * Adding relation onto.inst106_1 ts onto.inst106_1
* Owlready * Adding relation onto.inst173_0 tsr onto.inst173_0
* Owlready * Adding relation onto.inst173_0 tsr onto.inst173_2
* Owlready * Adding relation onto.inst173_0 ts onto.inst173_0
* Owlready * Adding relation onto.inst173_0 ts onto.inst173_2
* Owlready * Adding relation onto.inst178_2 tsr onto.inst178_2
* Owlready * Adding relation onto.inst178_2 ts onto.inst178_2
* Owlready * Adding relation onto.inst159_1 tsr onto.inst159_1
* Owlready * Adding relation onto.inst159_1 ts onto.inst159_1
* Owlre

* Owlready2 * Pellet took 1.6326508522033691 seconds
* Owlready2 * Pellet output:

 http://www.w3.org/2002/07/owl#Thing
    onto.owl#Believe_Class1
    onto.owl#Believe_Class2
    onto.owl#Believe_Class3
    onto.owl#Class1 - (onto.owl#inst73_1, onto.owl#inst208_0, onto.owl#inst30_1, onto.owl#inst129_1, onto.owl#inst106_1, onto.owl#inst173_0, onto.owl#inst178_2, onto.owl#inst159_1, onto.owl#inst28_0, onto.owl#inst217_1, onto.owl#inst19_0, onto.owl#inst158_0, onto.owl#inst157_2, onto.owl#inst83_1, onto.owl#inst30_2, onto.owl#inst185_1, onto.owl#inst212_1, onto.owl#inst113_2, onto.owl#inst21_1, onto.owl#inst85_1, onto.owl#inst210_1, onto.owl#inst141_1, onto.owl#inst225_1, onto.owl#inst205_2, onto.owl#inst68_0, onto.owl#inst192_2, onto.owl#inst2_0, onto.owl#inst199_0, onto.owl#inst139_1, onto.owl#inst156_2, onto.owl#inst146_1, onto.owl#inst10_1, onto.owl#inst37_2, onto.owl#inst119_1, onto.owl#inst175_1, onto.owl#inst152_2, onto.owl#inst211_2, onto.owl#inst147_0, onto.owl#inst171_0, onto.o

In [9]:
with onto:
  for inst in onto.individuals():
    close_world(inst)

onto.save('onto_reasoned.owl', format='ntriples')

In [10]:
with onto:
  sync_reasoner_pellet(debug = 2)

onto.save('onto_reasoned.owl', format='ntriples')

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /usr/local/lib/python3.8/dist-packages/owlready2/pellet/log4j-1.2.16.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/pellet-2.3.1.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/slf4j-log4j12-1.6.4.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-iri-0.9.5.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/aterm-java-1.6.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/httpcore-4.2.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-core-2.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/xml-apis-1.4.01.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/httpclient-4.2.3.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/antlr-3.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/antlr-runtime-3.2.jar:/usr/local/lib/python3.8/dist-packages